# Data Wrangle  - Data Audit and Clean 

### Downloaded the file from https://www.openstreetmap.org/relation/324211 

### 1. Creating the Sample Toronto OSM file.

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "toronto_canada.osm"  # Replace this with your osm file
SAMPLE_FILE = "torontosample.osm"

k = 15 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')


### 2. Audit the Data

In [1]:
"""
It should return a dictionary with the tag name as the key and number of times this tag can be encountered in 
the map as value.

"""
import xml.etree.cElementTree as ET
import pprint

def count_tags(filename):
    dict_ = {}
    for event,elem in ET.iterparse(filename):
        if elem.tag not in dict_:
            dict_[elem.tag] = 1
        else:
            dict_[elem.tag] += 1
    return dict_

def test():
    tags = count_tags('toronto_canada.osm')
    #tags = count_tags('torontosample.osm')
    pprint.pprint(tags) 

if __name__ == "__main__":
    test()

{'bounds': 1,
 'member': 151372,
 'nd': 5875055,
 'node': 5127518,
 'osm': 1,
 'relation': 9677,
 'tag': 5030403,
 'way': 761285}


In [2]:
"""
From lecture I have 3 regular expressions to check for certain patterns in the tags.
I would like to change the data model and expand the "addr:street" type of keys to a dictionary like this:
{"address": {"street": "Some value"}}

So, I want to see. I have such tags, and if I have any tags with problematic characters.

I have a count of each of four tag categories in a dictionary:
1.  "lower"        - for tags that contain only lowercase letters and are valid
2.  "lower_colon"  - for otherwise valid tags with a colon in their names
3.  "problemchars" - for tags with problematic characters 
4.  "other"        - for other tags that do not fall into the other three categories.

"""
import xml.etree.cElementTree as ET
import pprint
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

osm_file = "toronto_canada.osm"
#osm_file = "torontosample.osm"

def key_type(element, keys):
    if element.tag == "tag":
        att = element.attrib['k'] 
        m = lower.search(att)
        p = problemchars.search(att)
        n = lower_colon.search(att)
        if m:
            keys["lower"] += 1
        elif n:
            keys["lower_colon"] += 1
        elif p:
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
#pass    
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

def test():
    #keys = process_map('torontosample.osm')
    keys = process_map(osm_file)
    pprint.pprint(keys)

if __name__ == "__main__":
    test()

{'lower': 2970797,
 'lower_colon': 1939747,
 'other': 119222,
 'problemchars': 637}


In [ ]:
3. I am auditing and finding the abbreviated street name.

In [3]:
"""
The OSM file used is an abbreviated version of the Toronto Mapzen file  
"""

import xml.etree.cElementTree as ET
from collections import defaultdict
import re

#osm_file = open("torontosample.osm", "r")
osm_file = open("toronto_canada.osm", "r")
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())  # Python 2.7 ver
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

if __name__ == '__main__':
    audit()

#1: 2
#10: 1
#101: 1
#106: 1
#11: 2
#110: 2
#12A: 1
#17: 1
#18: 1
#185: 1
#19: 2
#2: 2
#200: 1
#202: 2
#204: 1
#23B: 1
#2801: 1
#282: 1
#2A: 1
#3: 1
#318: 1
#332: 1
#364: 1
#37: 1
#38: 1
#4: 4
#409: 1
#4B: 1
#5: 3
#5-6: 1
#57: 1
#6: 1
#7: 1
#8: 1
#A: 1
#A3: 1
#E8: 1
#G: 1
#PHW: 1
1: 329
10: 443
101: 4
102: 4
106: 2
107: 6
109: 19
10a: 4
11: 108
115c: 1
11a: 4
11b: 8
12: 139
124: 45
125: 12
12a: 4
12b: 4
13: 36
132: 2
13b: 4
14: 37
141: 2
146: 6
147: 4
14a: 4
14b: 6
14c: 2
14d: 4
14e: 2
14f: 2
15: 140
16: 60
17: 127
18: 120
19: 56
2: 517
20: 140
21: 76
22: 19
23: 36
24: 52
25: 159
26: 16
27: 379
28: 24
2b: 8
3: 387
30: 12
300: 1
302: 1
32: 52
34: 46
38: 1
39: 10
4: 359
400: 4
403: 2
42: 1
47: 98
48: 203
4b: 8
5: 283
50: 312
500: 1
52: 21
56: 2
57: 132
5700: 1
5a: 4
6: 459
6a: 6
7: 1190
7-12: 2
7;12: 5
7A: 84
8: 511
88: 53
89: 213
89): 1
8a: 4
9: 340
97: 32
99: 4
Abbey: 4
Access: 4
Acres: 14
Adjala: 180
Agostino: 4
Alley: 26
Alliston: 40
Amaranth: 42
Amberdale: 21
Ames: 8
Amici: 12
Anton

## 3. Imporving Stree Names, Postal Codes and Phone Numbers

In [5]:

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
 
OSM_FILE = "toronto_canada.osm"
#OSM_FILE = "torontosample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Terrace" ,"Expressway", "Crescent", "Close", "Common", "Heights", "Way"]

mapping = { "St": "Street",
            "St.": "Street",
            "ST": "Street",
            "Ben":"Bend",
            "Glenn":"Glen",
            "Hrbr":"Harbour",
            "Ho":"Hollow",
            "Mews":"Medows",
            "Pkwy":"Parkway",
            "Wood":"Woods",
            "Ave": "Avenue",
            "ave": "Avenue",
            "Rd.": "Road",
            "Blvd":"Boulevard",
            "Dr.":"Drive",
            "Ct":"Court",
            "Pl":"Place",
            "Sq":"Square",
            "ln":"Lane",
            "SW": "Southwest ",
            "SE": "Southeast ",
            "NW": "Northwest ",
            "NE": "Northeast ",
            'CT': 'Court',
            'Ct': 'Court',
            'Dr': 'Drive',
            'Dr.': 'Drive',
            'E': 'East',
            'Main St': 'Main Street',
            'N': 'North',
            'NE': 'Northeast',
            'NW': 'Northwest',
            'nw': 'Northwest',
            'PL': 'Place',
            'Pl': 'Place',
            'Rd': 'Road',
            'RD': 'Road',
            'Rd.': 'Road',
            'S': 'South',
            'S.': 'South',
            'SE': 'Southeast',
            'ST': 'Street',
            'SW': 'Southwest',
            'SW,': 'Southwest',
            'Se': 'Southeast',
            'southeast': 'Southeast',
            'St': 'Street',
            'st': 'Street',
            'Ter': 'Terrace',
            'W': 'West',
            'west': 'West',
            'HYW': 'Highway',
            'WY': 'Way',
            "Avebue":"Avenue",
            "Avenu":"Avenue"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            if street_type in mapping.keys():
                name = re.sub(street_type_re, mapping[street_type], name)
                               
    return name

#OSM_FILE_UPDATED = "toronto_canada1.osm"

# Mapped to the expected names, save/write the data to "toronto_canada1.osm" file 

OSM_FILE_UPDATED = "toronto_canada1.osm"
# Takes as input osm file and tuple of nodes and yield nodes of types from tuple. 

def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
# Following function will update the abbreviations in osm file

def update_street(original_file, update_file):
    with open(update_file, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ') 
        for i, element in enumerate(get_element(original_file)):
            for tag in element.iter("tag"):
                if is_street_name(tag):
                    tag.set('v',update_name(tag.attrib['v'], mapping))
            output.write(ET.tostring(element, encoding='utf-8'))
        output.write('</osm>')

update_street(OSM_FILE, OSM_FILE_UPDATED)


In [6]:

import xml.etree.cElementTree as ET
from collections import defaultdict
import re

postcode_type_re = re.compile(r'\d{5}-??')

def audit_post_type(post_types, zip):
    m = postcode_type_re.search(zip)
    if m:
        post_type = m.group()
        if post_type not in post_types:
            post_types[post_type].add(zip)
    else:
        post_types['unknown'].add(zip)

def is_pcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def postcode_audit(osmfile):
    osm_file = open(osmfile, "r")    
    post_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_pcode(tag):
                    audit_post_type(post_types, tag.attrib['v'])                
    osm_file.close()
    return post_types

postcode_types = postcode_audit(OSM_FILE_UPDATED)

pprint.pprint(dict(postcode_types))

OSM_FILE_UPDATED_PC = "toronto_canada2.osm"
# This function replace abbrevition by right zip
def u_postcode(zip):
    m = postcode_type_re.search(zip)
    if m:
        return m.group()
    else:
        return 'unknown'

# This function replace wrong zip in osm file
def update_postcode(original_file, update_file):
    with open(update_file, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ') 
        for i, element in enumerate(get_element(original_file)):
            for tag in element.iter("tag"):
                if is_pcode(tag):
                    tag.set('v',u_postcode(tag.attrib['v']))
            output.write(ET.tostring(element, encoding='utf-8'))
        output.write('</osm>')

update_postcode(OSM_FILE_UPDATED, OSM_FILE_UPDATED_PC)



{'14174': set(['14174']),
 '33913': set(['33913']),
 '96734': set(['96734']),
 'unknown': set(['B2W 5M5',
                 'B2Y4N4',
                 'K4A 1W9',
                 'L0B 1A0',
                 'L0B 1J0',
                 'L0B 1L0',
                 'L0C 1A0',
                 'L0C 1B0',
                 'L0C 1K0',
                 'L0E 1T0',
                 'L0G 1A0',
                 'L0G 1B0',
                 'L0G 1J0',
                 'L0G 1L0',
                 'L0G 1M0',
                 'L0G 1N0',
                 'L0G 1R0',
                 'L0G 1T0',
                 'L0G 1V0',
                 'L0G 1W0',
                 'L0G1A0',
                 'L0G1M0',
                 'L0G1T0',
                 'L0G1W0',
                 'L0H 1G0',
                 'L0J 1C0',
                 'L0L 1L0',
                 'L0M 1A0',
                 'L0N 1A0',
                 'L0N 1B0',
                 'L0N 1C0',
                 'L0N 1E0',
                 'L0N 1M0',
   

                 'L7L 5L1',
                 'L7L 5M4',
                 'L7L 5M9',
                 'L7L 5N3',
                 'L7L 5P9',
                 'L7L 5Y6',
                 'L7L 6A1',
                 'L7L 6A6',
                 'L7L 6C7',
                 'L7L 6M5',
                 'L7L 6M6',
                 'L7L 7H7',
                 'L7L 7L6',
                 'L7L1C7',
                 'L7L5H9',
                 'L7L7J8',
                 'L7M',
                 'L7M 0N6',
                 'L7M 0N9',
                 'L7M 0V5',
                 'L7M 0W3',
                 'L7M 0W4',
                 'L7M 1A9',
                 'L7M 1M2',
                 'L7M 4A6',
                 'L7M 4M8',
                 'L7M 5A5',
                 'L7M4A5',
                 'L7M4A6',
                 'L7M4A7',
                 'L7M4A9',
                 'L7M4B8',
                 'L7N 1M5',
                 'L7N 1M8',
                 'L7N 2G3',
                 'L7N 2M3',
    

                 'M6N 4R8',
                 'M6N 4Y9',
                 'M6N 4Z2',
                 'M6N 5C2',
                 'M6N 5H4',
                 'M6N5G4',
                 'M6P 0A1',
                 'M6P 0A6',
                 'M6P 1B2',
                 'M6P 1C9',
                 'M6P 1E2',
                 'M6P 1G5',
                 'M6P 1G7',
                 'M6P 1G8',
                 'M6P 1G9',
                 'M6P 1J5',
                 'M6P 1L5',
                 'M6P 1M6',
                 'M6P 1P3',
                 'M6P 1R3',
                 'M6P 1T8',
                 'M6P 1W7',
                 'M6P 1W9',
                 'M6P 1X2',
                 'M6P 1Y1',
                 'M6P 1Y5',
                 'M6P 1Y8',
                 'M6P 1Y9',
                 'M6P 1Z3',
                 'M6P 1Z4',
                 'M6P 1Z7',
                 'M6P 1Z8',
                 'M6P 2A4',
                 'M6P 2K6',
                 'M6P 2L5',
                 'M6P

In [7]:

# Compiler for cleaning phone format 
phone_type_re = re.compile(r'\d{3}\)?-?\s?.?\d{3}\s?-?\s?.?\d{4}')
phone_re = re.compile('\.|\)|\s|-')

def audit_phone_type(phone_types, phone):
    m = phone_type_re.search(phone)
    if m:
        phone_type = m.group()
        if phone_type not in phone_types:
            new_phone = phone_re.sub('',phone_type)
            new_phone = ('+1-' + new_phone[:3] + '-' +
                         new_phone[3:6] + '-' + new_phone[6:])
            phone_types[new_phone].add(phone)
    else:
        phone_types['unknown'].add(phone)
        
def is_phone(elem):
    return (elem.attrib['k'] == "phone")

def phone_audit(osmfile):
    osm_file = open(osmfile, "r")    
    phone_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone(tag):
                    audit_phone_type(phone_types, tag.attrib['v'])                
    osm_file.close()
    return phone_types
    
phones_types = phone_audit(OSM_FILE_UPDATED_PC)
# To validate or check the phone numbers format after the changes, uncomment the next line and comment the above one
#phones_types = phone_audit(OSM_FILE_UPDATED_PH) 

pprint.pprint(dict(phones_types))

OSM_FILE_UPDATED_PH = "toronto_canada3.osm"

# Following function update phone numbers to the correct format 

def u_phone(phone):
    m = phone_type_re.search(phone)
    if m:
        new_phone = phone_re.sub('', m.group())
        return ('+1-' + new_phone[:3] + '-' + new_phone[3:6] +
                '-' + new_phone[6:])        
    else:
        return phone

# This function replace the incosistent phone numbers with the right format

def update_phone(original_file, update_file):
    with open(update_file, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ') 
        for i, element in enumerate(get_element(original_file)):
            for tag in element.iter("tag"):
                if is_phone(tag):
                    tag.set('v',u_phone(tag.attrib['v']))
            output.write(ET.tostring(element, encoding='utf-8'))
        output.write('</osm>')

update_phone(OSM_FILE_UPDATED_PC, OSM_FILE_UPDATED_PH)
     


{'+1-128-993-89380': set(['+12899389380']),
 '+1-128-999-70644': set(['12899970644']),
 '+1-141-621-40008': set(['14162140008']),
 '+1-141-624-39362': set(['+14162439362']),
 '+1-141-625-11144': set(['+14162511144']),
 '+1-141-629-30404': set(['+14162930404']),
 '+1-141-629-87917': set(['+14162987917']),
 '+1-141-631-42901': set(['+14163142901']),
 '+1-141-635-17065': set(['+14163517065']),
 '+1-141-636-10331': set(['14163610331']),
 '+1-141-636-17454': set(['+14163617454']),
 '+1-141-636-40054': set(['+14163640054']),
 '+1-141-636-70633': set(['+14163670633']),
 '+1-141-639-72014': set(['+14163972014']),
 '+1-141-646-19887': set(['+14164619887']),
 '+1-141-646-34927': set(['+14164634927']),
 '+1-141-646-56434': set(['+14164656434']),
 '+1-141-650-42121': set(['+14165042121']),
 '+1-141-656-74576': set(['+14165674576']),
 '+1-141-659-59995': set(['14165959995']),
 '+1-141-662-32323': set(['+14166232323']),
 '+1-141-665-19999': set(['+14166519999']),
 '+1-141-667-45959': set(['+14166745

 '+1-416-251-9800': set(['416-251-9800']),
 '+1-416-252-0202': set(['416-252-0202']),
 '+1-416-252-0593': set(['+1 416 252-0593']),
 '+1-416-252-1288': set(['+1 416 252 1288']),
 '+1-416-252-1768': set(['416-252-1768']),
 '+1-416-252-1773': set(['+1 416-252-1773']),
 '+1-416-252-2002': set(['+1 416 252-2002']),
 '+1-416-252-2333': set(['+1 416 252-2333']),
 '+1-416-252-2825': set(['416-252-2825']),
 '+1-416-252-3665': set(['416-252-3665']),
 '+1-416-252-4272': set(['+1-416-252-4272']),
 '+1-416-252-4368': set(['416-252-4368']),
 '+1-416-252-4477': set(['+1 416 252-4477']),
 '+1-416-252-4868': set(['416-252-4868']),
 '+1-416-252-4900': set(['416-252-4900']),
 '+1-416-252-5205': set(['416-252-5205']),
 '+1-416-252-5695': set(['4162525695']),
 '+1-416-252-5821': set(['416-252-5821']),
 '+1-416-252-6466': set(['416-252-6466']),
 '+1-416-252-6822': set(['416-252-6822']),
 '+1-416-252-7061': set(['416-252-7061']),
 '+1-416-252-7222': set(['416-252-7222']),
 '+1-416-252-7266': set(['+1 416 25

 '+1-416-532-7222': set(['416-532-7222']),
 '+1-416-532-7701': set(['+1-416-532-7701']),
 '+1-416-532-9000': set(['416-532-9000']),
 '+1-416-533-0999': set(['+1 416 533 0999']),
 '+1-416-533-1155': set(['+1 416 5331155']),
 '+1-416-533-1221': set(['+1-416-533-1221']),
 '+1-416-533-2005': set(['+1 416-533-2005']),
 '+1-416-533-2088': set(['416-533-2088']),
 '+1-416-533-2341': set(['+1 416-533-2341']),
 '+1-416-533-2391': set(['416-533-2391']),
 '+1-416-533-3323': set(['4165333323']),
 '+1-416-533-5555': set(['+1 416 533 5555']),
 '+1-416-533-6066': set(['+1 416 533 6066']),
 '+1-416-533-6629': set(['+1-416-533-6629']),
 '+1-416-533-9168': set(['+1 416 533 9168']),
 '+1-416-533-9306': set(['416-533-9306']),
 '+1-416-534-1470': set(['416-534-1470']),
 '+1-416-534-2235': set(['+1-416-534-2235']),
 '+1-416-534-3432': set(['+1-416-534-3432']),
 '+1-416-534-4232': set(['+1 416 534 4232']),
 '+1-416-534-5914': set(['+1 416 534 5914']),
 '+1-416-535-0111': set(['4165350111']),
 '+1-416-535-0416

 '+1-416-902-8172': set(['416-902-8172']),
 '+1-416-906-7608': set(['+1 416-906-7608']),
 '+1-416-907-4460': set(['1 416 907 4460']),
 '+1-416-907-8074': set(['+1 416 907-8074']),
 '+1-416-907-8524': set(['416-907-8524']),
 '+1-416-907-9505': set(['+1 416 9079505']),
 '+1-416-909-6291': set(['416-909-6291']),
 '+1-416-910-4358': set(['+1 416 910 4358']),
 '+1-416-912-9994': set(['+1 416-912-9994']),
 '+1-416-913-0034': set(['+1 416 913 0034']),
 '+1-416-913-0200': set(['416-913-0200']),
 '+1-416-913-3132': set(['+1 416 913 3132']),
 '+1-416-913-4997': set(['416-913-4997']),
 '+1-416-913-7184': set(['+1 416 913 7184']),
 '+1-416-913-7296': set(['4169137296']),
 '+1-416-913-7345': set(['416-913-7345']),
 '+1-416-913-8880': set(['+1-416-913-8880']),
 '+1-416-915-5002': set(['+1 416 915 5002']),
 '+1-416-915-7777': set(['+1 416-915-7777']),
 '+1-416-916-0550': set(['416-916-0550']),
 '+1-416-916-2075': set(['+1 416-916-2075']),
 '+1-416-916-8811': set(['+1 416 916 8811']),
 '+1-416-918-657

 '+1-647-896-1774': set(['(647) 896-1774']),
 '+1-647-896-6987': set(['+1 647 896 6987']),
 '+1-647-931-6206': set(['+1 647 931 6206']),
 '+1-647-932-2004': set(['+1 647 932-2004']),
 '+1-647-955-3111': set(['+1 647 955 3111']),
 '+1-647-956-9101': set(['+1 647 956 9101']),
 '+1-647-968-5588': set(['647-968-5588']),
 '+1-647-969-0186': set(['+1 647-969-0186']),
 '+1-647-979-3333': set(['+1 647 979 3333']),
 '+1-647-980-6298': set(['647-980-6298']),
 '+1-647-984-7366': set(['647-984-7366']),
 '+1-647-985-3507': set(['647-985-3507']),
 '+1-647-985-5547': set(['+1 647-985-5547']),
 '+1-647-987-4227': set(['6479874227']),
 '+1-647-988-0262': set(['+1 647 988 0262']),
 '+1-647-989-3152': set(['+1 647 989 3152']),
 '+1-647-995-1465': set(['647-995-1465']),
 '+1-674-984-4855': set(['+1-674-984-4855']),
 '+1-705-434-1545': set(['+1-(705) 434-1545']),
 '+1-705-434-4381': set(['705-434-4381']),
 '+1-705-434-4763': set(['+1-(705) 434-4763']),
 '+1-705-434-9165': set(['+1-705-434-9165']),
 '+1-705

 '+1-905-420-8868': set(['905-420-8868']),
 '+1-905-421-9330': set(['905-421-9330']),
 '+1-905-425-1945': set(['+1-905-425-1945']),
 '+1-905-426-7772': set(['+1-905-426-7772']),
 '+1-905-426-8477': set(['905-426-8477']),
 '+1-905-427-4391': set(['905-427-4391']),
 '+1-905-427-5728': set(['+1-905-427-5728']),
 '+1-905-428-0310': set(['905-428-0310']),
 '+1-905-428-4556': set(['905-428-4556']),
 '+1-905-428-8600': set(['(905) 428-8600']),
 '+1-905-430-8400': set(['9054308400']),
 '+1-905-432-2066': set(['+1-905-432-2066']),
 '+1-905-432-3486': set(['+1-905-432-3486']),
 '+1-905-432-8350': set(['905-432-8350']),
 '+1-905-433-0011': set(['905-433-0011']),
 '+1-905-433-0211': set(['905-433-0211']),
 '+1-905-433-1600': set(['+1 905 433 1600']),
 '+1-905-433-4799': set(['905-433-4799']),
 '+1-905-433-5575': set(['+1 905 433 5575']),
 '+1-905-433-9191': set(['+1 905 433 9191']),
 '+1-905-434-2663': set(['905-434-2663']),
 '+1-905-434-5000': set(['+1 905-434-5000']),
 '+1-905-436-1704': set(['+

 '+1-905-639-3611': set(['(905) 639-3611']),
 '+1-905-639-3708': set(['+1 905 639-3708']),
 '+1-905-639-4327': set(['+1 905 639-4327']),
 '+1-905-639-4531': set(['+1-905-639-4531']),
 '+1-905-639-4622': set(['+1 905 639-4622']),
 '+1-905-639-5592': set(['+1 905 639-5592']),
 '+1-905-639-7001': set(['+1-905-639-7001']),
 '+1-905-639-7740': set(['905-639-7740']),
 '+1-905-639-9600': set(['905-639-9600']),
 '+1-905-640-0117': set(['+1 905 640 0117']),
 '+1-905-640-0691': set(['905-640-0691']),
 '+1-905-640-1140': set(['+1 905 640 1140']),
 '+1-905-640-1456': set(['+1-905-640-1456']),
 '+1-905-640-3092': set(['1-905-640-3092']),
 '+1-905-640-3633': set(['905-640-3633']),
 '+1-905-640-6449': set(['+1-905-640-6449']),
 '+1-905-640-6499': set(['+1 905 640 6499']),
 '+1-905-640-6954': set(['+1 905 640 6954']),
 '+1-905-640-6961': set(['+1-905-640-6961']),
 '+1-905-640-7482': set(['+1 905 640 7482']),
 '+1-905-642-1344': set(['+1-905-642-1344']),
 '+1-905-642-3275': set(['+1 905-642-3275']),
 '

 '+1-905-823-2000': set(['1 905 823 2000']),
 '+1-905-823-2626': set(['905 823 2626']),
 '+1-905-823-3900': set(['+1 905 823-3900']),
 '+1-905-823-4335': set(['905 823 4335']),
 '+1-905-823-4440': set(['1 905 823 4440']),
 '+1-905-823-4510': set(['905 823 4510']),
 '+1-905-823-5535': set(['905 823 5535']),
 '+1-905-823-5999': set(['1 905 823 5999']),
 '+1-905-823-6341': set(['1 905 823 6341']),
 '+1-905-823-6520': set(['+1 905 823 6520']),
 '+1-905-823-6878': set(['905 823 6878']),
 '+1-905-823-7600': set(['+1 905 823-7600']),
 '+1-905-823-7636': set(['+1 905 823-7636']),
 '+1-905-823-8430': set(['905 823 8430']),
 '+1-905-823-8787': set(['905 823 8787']),
 '+1-905-823-8818': set(['905-823-8818']),
 '+1-905-823-8988': set(['905-823-8988']),
 '+1-905-823-9252': set(['+1 905 823 9252']),
 '+1-905-823-9555': set(['905-823-9555']),
 '+1-905-824-0155': set(['+1 905 824-0155']),
 '+1-905-824-0360': set(['+1 905 824-0360']),
 '+1-905-824-0727': set(['+1 905 824-0727']),
 '+1-905-824-1025': se

 '+1-905-890-8990': set(['+1 905 890-8990']),
 '+1-905-890-9338': set(['+1 905 890-9338']),
 '+1-905-890-9797': set(['+1 905 890-9797']),
 '+1-905-891-0011': set(['905-891-0011']),
 '+1-905-891-0022': set(['+1 905 891 0022']),
 '+1-905-891-0111': set(['905-891-0111']),
 '+1-905-891-0510': set(['+1 905 891-0510']),
 '+1-905-891-0891': set(['9058910891']),
 '+1-905-891-1093': set(['+1 905 891 1093']),
 '+1-905-891-1279': set(['905-891-1279']),
 '+1-905-891-1377': set(['+1 905 891-1377']),
 '+1-905-891-1439': set(['+! 905 891-1439']),
 '+1-905-891-1523': set(['+1 905 891-1523']),
 '+1-905-891-1918': set(['1 905 891 1918']),
 '+1-905-891-2005': set(['+1 905 891-2005']),
 '+1-905-891-2288': set(['1 905 891 2288']),
 '+1-905-891-3333': set(['1 905 891 3333']),
 '+1-905-891-3338': set(['9058913338']),
 '+1-905-891-3647': set(['+1 905 891-3647']),
 '+1-905-891-3777': set(['+1 905-891-3777']),
 '+1-905-891-3825': set(['905-891-3825']),
 '+1-905-891-3865': set(['+1 905-891-3865']),
 '+1-905-891-

Seeing abnormal or not regular format phone numbers as following

 'unknown': set(['(416) 536-SODA',
                 '+1 905 -90-4110',
                 '1 905 891 326',
                 '439-0000'])}

In [8]:
#The function process_map should return a set of unique user IDs ("uid")

import xml.etree.cElementTree as ET
import pprint
import re

def get_user(element):
    if element.get('uid'):
        return element.get('uid')
    
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        users.add(get_user(element))
        users.discard(None)
    return users

def test():

    users = process_map('toronto_canada3.osm')
    pprint.pprint(users)


if __name__ == "__main__":
    test()
    

set(['1',
     '100337',
     '100419',
     '1007022',
     '1007528',
     '100917',
     '1009527',
     '101184',
     '1012362',
     '1015527',
     '1015528',
     '1015531',
     '1015532',
     '1015536',
     '1015542',
     '102064',
     '10225',
     '102410',
     '102723',
     '103253',
     '1035032',
     '10353',
     '103530',
     '1035507',
     '103574',
     '103816',
     '1044834',
     '104519',
     '1051550',
     '105255',
     '105603',
     '105839',
     '1058666',
     '1060930',
     '106293',
     '1063003',
     '1066031',
     '106858',
     '106874',
     '106914',
     '1069176',
     '10716',
     '1074988',
     '10786',
     '10788',
     '107884',
     '1080971',
     '1083420',
     '108634',
     '1087647',
     '109002',
     '10927',
     '1093914',
     '109925',
     '110046',
     '110253',
     '110263',
     '1103322',
     '1103744',
     '1106095',
     '1108251',
     '110915',
     '1110934',
     '1114651',
     '111504',
     '

     '411528',
     '411561',
     '4120008',
     '41214',
     '412154',
     '4129036',
     '4129101',
     '413360',
     '4138265',
     '4138390',
     '413853',
     '414538',
     '4148813',
     '415250',
     '4153291',
     '416346',
     '4167103',
     '4174894',
     '4178914',
     '4178994',
     '418347',
     '4190651',
     '419113',
     '4193842',
     '419601',
     '4208231',
     '4210501',
     '42123',
     '4213567',
     '4215568',
     '4216140',
     '42191',
     '4221303',
     '4225057',
     '4225751',
     '4231697',
     '4240592',
     '42429',
     '4250350',
     '425214',
     '4254864',
     '4255535',
     '425846',
     '4258748',
     '4260390',
     '427384',
     '4274282',
     '4276249',
     '428339',
     '428720',
     '4288229',
     '4293558',
     '4294297',
     '4297322',
     '4302385',
     '4304988',
     '4308356',
     '43087',
     '4315127',
     '4315320',
     '4320188',
     '4320689',
     '4324046',
     '4324792',
  

     '5666377',
     '5666698',
     '5666860',
     '5667446',
     '5667994',
     '5668280',
     '5669016',
     '5669281',
     '5669613',
     '567139',
     '5671966',
     '5674088',
     '567601',
     '5677293',
     '567792',
     '5682957',
     '5683437',
     '5686313',
     '5687126',
     '56884',
     '5688958',
     '5692508',
     '5692965',
     '5697319',
     '5697688',
     '5698967',
     '5700381',
     '5703022',
     '570774',
     '571080',
     '571170',
     '5715057',
     '5715561',
     '57158',
     '5722011',
     '5725028',
     '5731246',
     '5737518',
     '5743216',
     '574654',
     '574684',
     '574730',
     '574775',
     '574850',
     '5751301',
     '5751745',
     '5756206',
     '5761375',
     '576174',
     '576338',
     '5764933',
     '5765055',
     '5765492',
     '5766343',
     '576659',
     '576728',
     '5768886',
     '5769800',
     '5771196',
     '5771543',
     '577490',
     '577807',
     '578519',
     '5788493'

### 4. Prepair for Database SQL 

After auditing is complete the next step is to prepare the data to be inserted into a SQL database.
   To do so I will parse the elements in the OSM XML file, transforming them from document format to
   tabular format.

   Making it possible to write to .csv files.  These csv files can then easily be imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

#### From the Case study

I've already provided the code needed to load the data, perform iterative parsing and write the
output to csv files. My task is to complete the shape_element function that will transform each
element into the correct format. To make this process easier we've already defined a schema (schema.py)
for the .csv files and the eventual tables. Using the cerberus library we can validate the output
against this schema to ensure it is correct.

Shape Element Function
The function should take as input an iterparse Element object and return a dictionary.

If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
import sys
import schema

OSM_PATH = "toronto_canada3.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  

    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        
        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':',1)[0]
                node_tag['key'] = child.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']):
                continue
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child.attrib['k']
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]
        
        position = 0
        for child in element:
            way_tag = {}
            way_node = {}
            
            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']):
                    continue
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                    
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    #process_map(OSM_PATH, validate=False)
    process_map(OSM_PATH, validate=True)
